In [ ]:
!pip install pandas
!pip install openpyxl
!pip install matplotlib
!pip install seaborn
!pip install scikit-learn

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Daten laden
data = pd.read_excel('SpotifyTopSongs_1000.xlsx')

# Ziele und Merkmale definieren
X = data.drop(columns=['popularity'])  # Alle Spalten außer 'popularity'
y = data['popularity']  # Zielvariable

# Datensatz in Train- und Test-Sätze aufteilen
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# Modell initialisieren
model = RandomForestRegressor(random_state=42)

# Modell trainieren
model.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Vorhersagen für den Testdatensatz
y_pred = model.predict(X_test)

# Fehler metrischen
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)  # Wurzel des MSE
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"R² Score: {r2:.2f}")

"""
### Dokumentation

#### 1. Datensatz aufteilen
Der Datensatz wurde in einen Trainingsdatensatz (80%) und einen Testdatensatz (20%) aufgeteilt.

#### 2. Algorithmus auswählen
Als Algorithmus wurde der **Random Forest Regressor** gewählt, da er robust ist und komplexe Beziehungen zwischen den Merkmalen und dem Ziel erfassen kann.

#### 3. Modell evaluieren
- **MAE:** {mae:.2f}
- **MSE:** {mse:.2f}
- **RMSE:** {rmse:.2f}
- **R² Score:** {r2:.2f}

#### 4. Schlussfolgerungen
Das Modell erreicht einen R²-Score von {r2:.2f}, was darauf hinweist, dass es {r2*100:.2f}% der Varianz im Zielattribut erklärt. Der MAE beträgt {mae:.2f}.
"""